In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import f1_score

2023-05-30 15:39:35.452994: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load the dataset
data_path = '/Users/as274094/GitHub/Refractored_star_classifier/tensorflow_version/'
dataset = np.load(data_path + 'PCA_dataset1.npy', allow_pickle=True)[()]

x_train = dataset['train_stars_pca']
x_val = dataset['validation_stars_pca']
x_test = dataset['test_stars_pca']
y_train = dataset['train_C']
y_val = dataset['validation_C']
y_test = dataset['test_C']
SED_test = dataset['test_SEDs']


In [3]:
# Hyperparameters
PCA_components = 24
model_learning_rate = 0.1
N_epochs = 50
N_committee = 48

In [4]:
def SEDlisttoC(SED_list):
    sed_array = np.array(SED_list)
    return sed_array*0.5 + 1.5

def CtoSEDarray(c_values, variance):
    sed_classes = ((c_values - 1.25) // 0.5).astype(int)
    sed_classes = np.where((c_values < 1.25) | (c_values > 7.75), 20, sed_classes)
    sed_classes = np.where((variance > 1.00), 20, sed_classes)
    return sed_classes

def calculate_success_rate(confusion_matrix):
    diagonal = np.trace(confusion_matrix)
    diagonal_neighbors = np.sum(np.diagonal(confusion_matrix, offset=1)) + np.sum(np.diagonal(confusion_matrix, offset=-1))
    total_classified = np.sum(confusion_matrix)
    
    success_rate = (diagonal + diagonal_neighbors) / total_classified
    return success_rate

In [12]:
# Define the model architecture

initializer = tf.keras.initializers.GlorotNormal(seed = 1)
model = tf.keras.Sequential([
    layers.Dense(26, input_shape=[PCA_components], activation='sigmoid', kernel_initializer= initializer),
    layers.Dense(26, activation='sigmoid', kernel_initializer= initializer),
    layers.Dense(1, activation = 'linear', kernel_initializer= initializer)
])
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.Adam(learning_rate = model_learning_rate)
)

# Train the model and make predictions
model.fit(x_train, y_train, epochs= N_epochs, verbose=0, validation_data=(x_val,y_val))
C_pred = model.predict(x_test, verbose = 1).reshape(-1) # Predict the scalar parameter C using the committee    

y_pred = np.mean(C_pred)
SED_pred = CtoSEDarray(C_pred,np.zeros_like(C_pred))


1/1 [==============================] - 0s 67ms/step


In [13]:
print(C_pred)
print(y_pred)
print(pred_variance)
print(SED_pred)

[4.60003   4.6000295 4.6000304 4.6000295 4.6000295 4.6000295 4.6000295
 4.6000304 4.6000295 4.600029 ]
4.60003
2.7284842e-13
[6 6 6 6 6 6 6 6 6 6]


In [17]:
# Evaluate the performance of the committee

mse = np.mean((y_test - y_pred)**2)
print('MSE:', mse)
print("Variance: ", pred_variance)

f1 = f1_score(SED_test, SED_pred, average = None)
print('\nF1 score:', f1)
print(np.mean(f1[:13]))

for i in range(len(SED_test)):
    print(SED_test[i], C_pred[i], SED_pred[i])


confusion_matrix = tf.math.confusion_matrix(SED_test, SED_pred) 
print("\nConfusion matrix:")
print(confusion_matrix)

success_rate = calculate_success_rate(confusion_matrix)
print('\nSuccess rate:', success_rate)



MSE: 2.5850209626582
Variance:  2.7284842e-13

F1 score: [0.         0.         0.         0.18181818 0.         0.
 0.         0.        ]
0.022727272727272728
0 4.60003 6
11 4.6000295 6
4 4.6000304 6
7 4.6000295 6
6 4.6000295 6
3 4.6000295 6
3 4.6000295 6
4 4.6000304 6
9 4.6000295 6
8 4.600029 6

Confusion matrix:
tf.Tensor(
[[0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 2 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]], shape=(12, 12), dtype=int32)

Success rate: 0.2
